<a href="https://colab.research.google.com/github/Kolo-Naukowe-Axion/QC1/blob/dnn/DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
!pip install ucimlrepo
from ucimlrepo import fetch_ucirepo
import numpy as np

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cpu


Taking data, conversion to numpy

In [12]:
banknote_authentication = fetch_ucirepo(id=267)

X_raw = banknote_authentication.data.features
y_raw = banknote_authentication.data.targets

X = X_raw.values
y = y_raw.values.ravel()

Test and train split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# stadarization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

In [15]:
class BanknoteDNN(nn.Module):
    def __init__(self):
        super(BanknoteDNN, self).__init__()
        self.layer1 = nn.Linear(4, 64)
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 16)
        self.output = nn.Linear(16, 1)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.dropout(x)
        x = self.relu(self.layer2(x))
        x = self.relu(self.layer3(x))
        x = self.sigmoid(self.output(x))
        return x

model = BanknoteDNN().to(device)

In [19]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 50
print("\nStarting training...")

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

model.eval()
with torch.no_grad():
    y_pred_probs = model(X_test_tensor)
    y_pred_class = (y_pred_probs > 0.5).float() # if > 0.5 fake

    correct = (y_pred_class == y_test_tensor).sum().item()
    acc = correct / len(y_test)
    print(f"\nAccuracy test set: {acc*100:.2f}%")


Starting training...
Epoch [10/50], Loss: 0.0000
Epoch [20/50], Loss: 0.0041
Epoch [30/50], Loss: 0.0006
Epoch [40/50], Loss: 0.0001
Epoch [50/50], Loss: 0.0001

Accuracy test set: 100.00%


In [23]:
!git clone https://github.com/Kolo-Naukowe-Axion/QC1.git

fatal: destination path 'QC1' already exists and is not an empty directory.
